# Running the strategy

In [1]:
! python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_2 \
--mode=backtest \
--tickers=EUR_USD,GBP_USD \
--ml_log=True

Strategy run finished with Run ID: 4
ML Log Saved in Minio Bucket: model-support-files as 4_ml_log.csv
Profit ... or Loss: 32.62


In [1]:
! python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_2 \
--mode=backtest \
--tickers=EUR_USD,GBP_USD,AUD_USD,EUR_JPY,NZD_USD \
--ml_log=True

Strategy run finished with Run ID: 20
ML Log Saved in Minio Bucket: model-support-files as 20_ml_log.csv
Profit ... or Loss: 79.46


In [ ]:
! python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_1 \
--mode=backtest \
--tickers=EUR_USD,GBP_USD,AUD_USD,EUR_JPY,NZD_USD \
--ml_log=True

In [1]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
import io
import boto3

# Preprocessing the file

In [2]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
data = pd.read_csv(s3.get_object(Bucket="model-support-files", Key='20_ml_log.csv')['Body'],sep=',')
data.head(10)

,security,datetime,close,RSI,STOCHASTIC,ATR
0,EUR_USD,2010-12-30 22:00:00,1.32855,NaN,NaN,NaN
1,EUR_USD,2010-12-31 22:00:00,1.33815,NaN,NaN,NaN
2,EUR_USD,2011-01-01 22:00:00,1.33280,NaN,NaN,NaN
3,EUR_USD,2011-01-02 22:00:00,1.32512,NaN,NaN,NaN
4,EUR_USD,2011-01-03 22:00:00,1.32919,NaN,NaN,NaN
5,EUR_USD,2011-01-04 22:00:00,1.31258,NaN,NaN,0.007796
6,EUR_USD,2011-01-05 22:00:00,1.29978,NaN,NaN,0.007135
7,EUR_USD,2011-01-06 22:00:00,1.29057,NaN,NaN,0.006176
8,EUR_USD,2011-01-08 22:00:00,1.28607,NaN,NaN,0.005193
9,EUR_USD,2011-01-09 22:00:00,1.28742,NaN,NaN,0.006228


In [3]:
from ml_pack.preprocessing.ml_preprocessing import ml_preprocessing
preprocessed_file=ml_preprocessing(input_file="20_ml_log.csv",fwd_returns=5)
preprocessed_file

'processed_20_ml_log.csv'

In [4]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
data = pd.read_csv(s3.get_object(Bucket="model-support-files", Key=preprocessed_file)['Body'],sep=',')

In [5]:
data.head(10)

,RSI,STOCHASTIC,fwd_returns
0,56.991619,78.882076,-0.005146
1,49.073894,60.403681,0.005311
2,60.432136,61.626107,-0.006642
3,59.448606,77.279931,0.003864
4,50.438378,54.132662,0.005214
5,43.262870,13.615443,0.008839
6,61.668242,58.322696,-0.005078
7,50.683722,54.940994,0.001466
8,46.870835,32.905598,0.017321
9,62.520782,56.724459,0.006251


# Train a ML-model using MLflow

### Point to the MLflow tracking endpoint

In [6]:
mlflow.tracking.set_tracking_uri('http://mlflow-image:5500')

### Create a MLflow experiment

In [7]:
# mlflow.delete_experiment(1)

In [8]:
# mlflow.create_experiment(name='simple_trading_models', artifact_location='s3://mlflow-models')

In [9]:
mlflow.set_experiment('simple_trading_models')

### Fetching the preprocessed data from Minio

In [10]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

warnings.filterwarnings("ignore")
np.random.seed(40)
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "fwd_returns"
train_x = train.drop(["fwd_returns"], axis=1)
test_x = test.drop(["fwd_returns"], axis=1)
train_y = train[["fwd_returns"]]
test_y = test[["fwd_returns"]]

n_estimators=300
max_depth=10

lr = RandomForestRegressor(n_estimators=n_estimators,max_depth=max_depth)
lr.fit(train_x, train_y)
predicted_qualities = lr.predict(test_x)
(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("RandomForest Model (n_estimators=%f, max_depth=%f):" % (n_estimators, max_depth))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

mlflow.log_param("n_estimators", n_estimators)
mlflow.log_param("max_depth", max_depth)
mlflow.log_metric("rmse", rmse)
mlflow.log_metric("r2", r2)
mlflow.log_metric("mae", mae)
mlflow.sklearn.log_model(lr, "model")

RandomForest Model (n_estimators=300.000000, max_depth=10.000000):
  RMSE: 0.009809113217502488
  MAE: 0.007529644150039793
  R2: 0.43336618232607704


## Model Serving

In [11]:
import mlflow.pyfunc

In [14]:
model_predict=mlflow.pyfunc.load_model(model_uri="s3://mlflow-models/0255c7673c164de6815d89a4228b846c/artifacts/model")

In [15]:
# model_predict=mlflow.pyfunc.load_model(model_uri="s3://mlflow-models/0255c7673c164de6815d89a4228b846c/model") 

In [16]:
model_predict.predict([[80,20]])

array([0.00872057])

## Running the strategy with the model

In [17]:
!python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_2 \
--strat_param=ml_serving=True,model_uri=0255c7673c164de6815d89a4228b846c \
--ml_log=False \
--mode=backtest \
--tickers=EUR_USD,GBP_USD,NZD_USD

s3://mlflow-models/0255c7673c164de6815d89a4228b846c/artifacts/model
Strategy run finished with Run ID: 21
Profit ... or Loss: -1262.76


## Packaging the model using MLflow (BONUS)
For reproducibility and for sharing

In [55]:
mlflow.projects.run("/home/jovyan/work/BT/mlflow_project",parameters={'n_estimators':200,'max_depth':10})


## Serving the model as a rest API using MLflow (BONUS)
Serrve the model in the mlflow tracking container using

docker exec ekholabs-mlflow /bin/sh -c "mlflow models serve -m /ekholabs-mlflow/mlruns/0/a85ab97a393045afaea2b550a79686e8/artifacts/model --host=0.0.0.0 -p 2349"

Then you can call in the program using curl

In [ ]:
! curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 2, 10, 0.98, 1, 45, 2, 1.2, 44, 4, 0.66]]}' http://mlflow-image:2349/invocations

In [22]:
! python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_2 \
--mode=backtest \
--tickers=EUR_USD,GBP_USD \
--strat_param=model_uri=b5c9975d81c64bc3b662654007e1c20d,ml_serving=True

s3://mlflow-models/b5c9975d81c64bc3b662654007e1c20d/artifacts/model
Profit ... or Loss: -969.07


In [5]:
! python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_2\
--mode=backtest\
--tickers=EUR_USD,GBP_USD,JPY_USD \
--strat_param=model_uri=0255c7673c164de6815d89a4228b846c,ml_serving=True

s3://mlflow-models/0255c7673c164de6815d89a4228b846c/artifacts/model
Traceback (most recent call last):
  File "/home/jovyan/work/q_pack/q_run/run_BT.py", line 130, in <module>
    run()
  File "/home/jovyan/work/q_pack/q_run/run_BT.py", line 70, in run
    results = cerebro.run(tradehistory=True)  
  File "/opt/conda/lib/python3.7/site-packages/backtrader/cerebro.py", line 1127, in run
    runstrat = self.runstrategies(iterstrat)
  File "/opt/conda/lib/python3.7/site-packages/backtrader/cerebro.py", line 1293, in runstrategies
    self._runonce(runstrats)
  File "/opt/conda/lib/python3.7/site-packages/backtrader/cerebro.py", line 1652, in _runonce
    strat._once()
  File "/opt/conda/lib/python3.7/site-packages/backtrader/lineiterator.py", line 297, in _once
    indicator._once()
  File "/opt/conda/lib/python3.7/site-packages/backtrader/lineiterator.py", line 297, in _once
    indicator._once()
  File "/opt/conda/lib/python3.7/site-packages/backtrader/lineiterator.py", line 297, in _on

In [ ]:
,NZD_USD,